# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load the csv file
df = pd.read_csv('../my_output_data/city_info.csv')
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.12,-134.97,73.20,79,0,14.09,PF,1602429512
1,Esperance,-33.87,121.90,51.80,100,75,4.70,AU,1602429512
2,Puerto del Rosario,28.50,-13.86,77.00,78,20,14.99,ES,1602429512
3,Mataura,-46.19,168.86,53.01,92,79,10.00,NZ,1602429512
4,Beringovskiy,63.05,179.32,30.78,97,99,18.30,RU,1602429513
...,...,...,...,...,...,...,...,...,...
554,Lata,40.16,-8.33,72.00,35,0,8.01,PT,1602429595
555,Avera,33.19,-82.53,79.00,88,75,11.41,US,1602429595
556,Ushuaia,-54.80,-68.30,44.02,75,82,6.40,AR,1602429529
557,Punta Arenas,-53.15,-70.92,53.01,62,20,38.03,CL,1602429269


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# enable the display gmap
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [8]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
#set up the figure layout
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#create the empyty list to store the data generated
coordinates=[]
hums = []
# for loop to convert the df to the list
for index, row in df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    hum = row['Humidity']
    coordinate = (lat,lng)
    coordinates.append(coordinate)
    hums.append(hum)
#load the figure_layout
fig = gmaps.figure(layout=figure_layout)
#set up the heatmap layer based on the coordinates and humunity
heatmap = gmaps.heatmap_layer(coordinates,weights=hums,dissipating=True,max_intensity = 100)
#add the heatlayer
fig.add_layer(heatmap)
#show the fig
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# set up the filt for the selected cities
filt = (df['Max Temp']>70)&(df['Max Temp']<90)&(df['Cloudiness']==0)&(df['Wind Speed']<10)
# use loc to filt the original df and generate the selected_df
selected_df = df.loc[filt]
selected_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,Dafeng,23.45,108.59,71.67,74,0,3.36,CN,1602429528
85,Riyadh,24.69,46.72,89.60,19,0,4.70,SA,1602429399
205,Buraidah,26.33,43.98,86.00,17,0,5.82,SA,1602429344
240,Najrān,17.49,44.13,86.00,23,0,4.70,SA,1602429554
385,San Vicente,-26.62,-54.13,86.34,19,0,3.58,AR,1602429573
511,Marsaxlokk,35.84,14.54,75.20,60,0,9.17,MT,1602429589
554,Lata,40.16,-8.33,72.00,35,0,8.01,PT,1602429595


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Store into variable named hotel_df.
hotel_df=pd.DataFrame()
#set up the parameters for nearbysearch
# target_search = "hotel"
target_radius = 5000
target_type = "lodging"
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#set up params
params = {
#     "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
# set up the empyty list
hotel_names=[]
city_names=[]
country_names=[]
lats=[]
lngs=[]
#for loop to get the row data
for index, row in selected_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    coordinate=(lat,lng)
    # try to ger the city info
    try:
        places_data = requests.get(base_url, params=params).json()
        hotel_name=places_data['results'][0]["name"]
        hotel_names.append(hotel_name)
        city_names.append(row['City'])
        country_names.append(row['Country'])
        coordinates.append(coordinate)
        lats.append(lat)
        lngs.append(lng)
    except (IndexError):
        pass
# load the collected data to the df
hotel_df['Hotel Name']=hotel_names
hotel_df['City']=city_names
hotel_df['Country']=country_names
hotel_df['Lat']=lats
hotel_df['Lng']=lngs
hotel_df


,Hotel Name,City,Country,Lat,Lng
0,Shenglong Hotel,Dafeng,CN,23.45,108.59
1,InterContinental Riyadh,Riyadh,SA,24.69,46.72
2,Mövenpick Hotel Qassim,Buraidah,SA,26.33,43.98
3,محمد هادي ال فطيح,Najrān,SA,17.49,44.13
4,Complejo Turístico Farelluk,San Vicente,AR,-26.62,-54.13
5,Water's Edge,Marsaxlokk,MT,35.84,14.54
6,Casa Maquia,Lata,PT,40.16,-8.33


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(marker_layer)
fig.add_layer(heatmap)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…